## InceptionNet Basic

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, MaxPool2D, AveragePooling2D, Flatten, concatenate, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical

from PIL import Image
import cv2
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder

2023-09-16 21:43:56.318686: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-16 21:43:56.506430: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-16 21:43:56.508103: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-16 21:43:57.532460: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load Image
def load_images(folder_path):
    X = []
    y = []
    valid_extensions = ['.jpg', '.jpeg', '.png']

    img_folders = sorted(os.listdir(folder_path))

    for folder_name in img_folders:
        image_folder_path = os.path.join(folder_path,folder_name)
        images_path = os.listdir(image_folder_path)
        for image_path in images_path:
            file_extension = os.path.splitext(image_path)[-1].lower()
            if file_extension in valid_extensions:
                image_path = os.path.join(image_folder_path, image_path)
                image = Image.open(image_path)
                image = image.resize((224,224)) 
                image = np.array(image)
                # Or
                # image = cv2.imread(image_path)
                # image = image.resize((224,224))
                # image = np.array(image)
                X.append(image)
                y.append(os.path.basename(folder_name))

    return np.array(X), np.array(y)

In [3]:
img_folder_path = 'artifacts/17flowers/jpg'
X, y = load_images(img_folder_path)

In [4]:
y

array(['Bluebell', 'Bluebell', 'Bluebell', ..., 'Violet', 'Violet',
       'Violet'], dtype='<U10')

In [5]:
X.shape

(1360, 224, 224, 3)

In [6]:
y.shape

(1360,)

In [7]:
x_train = X.astype('float32') / 255.0

In [8]:
x_train.shape

(1360, 224, 224, 3)

In [9]:
label_encoder = LabelEncoder()
integer_labels = label_encoder.fit_transform(y)

In [10]:
integer_labels

array([ 0,  0,  0, ..., 16, 16, 16])

In [11]:
y_train = to_categorical(integer_labels, num_classes=17)

In [12]:
y_train.shape

(1360, 17)

In [13]:
# Make Inception Block
# https://www.analyticsvidhya.com/blog/2018/10/understanding-inception-network-from-scratch/

def inception_block(x, filters):
    tower_1 = Conv2D(filters[0], (1,1), padding='same', activation='relu')(x)
    tower_1 = Conv2D(filters[1], (3,3), padding='same', activation='relu')(tower_1)

    tower_2 = Conv2D(filters[2], (1,1), padding='same', activation='relu')(x)
    tower_2 = Conv2D(filters[3], (5,5), padding='same', activation='relu')(tower_2)

    tower_3 = MaxPool2D((3,3), padding='same', strides=(1,1))(x)
    tower_3 = Conv2D(filters[4], (1,1), padding='same', activation='relu')(tower_3)
    
    output = concatenate([tower_1, tower_2, tower_3], axis=3)
    return output
    

In [14]:
# Make Inception Model
def inception(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    x = Conv2D(64, (3,3), padding='same', activation='relu')(inputs)
    x = MaxPool2D((2,2))(x)

    x = inception_block(x, filters=[64,96,128,16,32])
    x = inception_block(x, filters=[128,128,192,32,96])
    x = MaxPool2D((2,2))(x)

    x = inception_block(x, filters=[192,96,208,16,48])
    x = inception_block(x, filters=[160,112,224,24,64])
    x = inception_block(x, filters=[128,128,256,24,64])
    x = inception_block(x, filters=[112,144,288,32,64])
    x = MaxPool2D((2,2))(x)

    x = inception_block(x, filters=[256,160,320,32,128])
    x = inception_block(x, filters=[256,160,320,32,128])
    x = inception_block(x, filters=[384,192,384,48,128])

    x = AveragePooling2D((4,4))(x)
    x = Flatten()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model    

In [15]:
# Create the Inception Model
model = inception(input_shape=(224,224,3), num_classes=17)

In [16]:
# Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Print the Summery of the Model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 224, 224, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)         0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 112, 112, 64)         4160      ['max_pooling2d[0][0]']   

In [ ]:
# Train the model
history = model.fit(x_train, y_train, batch_size=40, epochs=5, verbose=1, validation_split=0.2, shuffle=True)

Epoch 1/5
28/28 [==============================] - 815s 29s/step - loss: 2.7009 - accuracy: 0.0818 - val_loss: 22.8555 - val_accuracy: 0.0000e+00
Epoch 2/5
28/28 [==============================] - 787s 28s/step - loss: 2.6948 - accuracy: 0.0800 - val_loss: 5.7992 - val_accuracy: 0.0000e+00
Epoch 3/5
